In [1]:
library('SAVER')

# saver_obj = readRDS('/data_volume/memento/saver/melanoma_dropseq_saver.rds')

# dropseq_data = readRDS('melanoma_dropseq.rds')

# saver_obj = saver(dropseq_data, ncores=3)

# saveRDS(saver_obj, 'saver_melanoma.rds')

# saver_obj <- readRDS('saver_melanoma.rds')

# write.csv(saver_obj[['estimate']], '/data_volume/memento/saver/saver_estimates.csv')

In [2]:
suppressMessages(library(Seurat))
suppressMessages(library(SeuratDisk))

In [3]:
DATA_PATH <- '/home/ubuntu/Data/'
setwd(paste(DATA_PATH, 'smfish/correlation/', sep=''))


In [4]:
fname <- '500_0'

In [5]:
idxs <- c(26973,
 12715,
 13655,
 8976,
 11611,
 8827,
 15880,
 7778,
 12686,
 30953,
 28386,
 9236,
 3718,
 31243,
 30532,
 4360,
 30984,
 15086,
 3158,
 11397,
 11612)+1

In [6]:
Convert(paste(fname, '.h5ad', sep=''), dest = "h5seurat", overwrite = TRUE, verbose = FALSE, misc=FALSE)
seurat <- LoadH5Seurat(paste(fname, '.h5seurat', sep=''), meta.data=FALSE,misc=FALSE, verbose = FALSE)

matrix = as.matrix(seurat$RNA$counts)
colnames(matrix) <- colnames(seurat$RNA)
rownames(matrix) <- rownames(seurat$RNA)

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Validating h5Seurat file



In [7]:
saver_obj = saver(matrix, ncores=10, pred.genes=idxs)


32287 genes, 500 cells

Running SAVER with 10 worker(s)

Calculating predictions for 21 genes using 3989 genes and 500 cells...

Start time: 2024-02-01 23:17:46.355533

Estimating finish time...

Finished 10/32287 genes. Approximate finish time: 2024-02-01 23:18:01.459194

Calculating max cor cutoff...

Finished 21/32287 genes. Approximate finish time: 2024-02-01 23:18:00.499599

Estimating remaining 32266 genes.

Done!

Finish time: 2024-02-01 23:18:08.53906

Total time: 22.18184 secs



### Read saved RDS file

saver_obj = readRDS('melanoma_dropseq_saver.rds')

corr_df = cor.genes(saver_obj)

dim(corr_df)

write.csv(corr_df, '/data_volume/memento/saver/saver_corrs.csv')

### Saver correlation code

In [8]:
sf <- colSums(matrix)/mean(colSums(matrix))

In [9]:
num_genes <-length(idxs)

In [10]:
library(reshape2)

adj.vec <- rep(0, num_genes)
for (i in 1:num_genes) {
  adj.vec[i] <- 
    sqrt(var(saver_obj$estimate[i, ]*sf, na.rm = TRUE)/
           (var(saver_obj$estimate[i, ]*sf, na.rm = TRUE) + 
              mean(saver_obj$se[i, ]^2*sf^2, na.rm = TRUE)))
}

In [ ]:
scale.factor <- outer(adj.vec, adj.vec)


temp.cor <- cor( t(sweep(saver_obj$estimate, 2, sf, "*")))

cor1 <- temp.cor*scale.factor
cor2 <- cor.genes(saver_obj)

In [29]:
dim(cor1)

[1] 32287 32287

In [34]:
cor1[idxs, idxs]

,gene26973,gene12715,gene13655,gene8976,gene11611,gene8827,gene15880,gene7778,gene12686,gene30953,⋯,gene9236,gene3718,gene31243,gene30532,gene4360,gene30984,gene15086,gene3158,gene11397,gene11612
gene26973,0.2550764961,0.0036459800,0.1347476064,0.076772279,0.1272769422,0.065841391,NA,-1.810441e-03,0.1726072263,0.116606712,⋯,0.166210883,0.1065269131,NA,0.1320843390,0.1196365805,0.1366383867,NA,1.539891e-01,-0.0007223674,0.155723029
gene12715,0.0036459800,0.2125014874,0.0180524702,0.010901894,0.0165878932,0.010203946,NA,-8.876778e-04,0.0223713578,0.013313234,⋯,0.018083205,0.0047636079,NA,0.0178197162,0.0191590529,0.0101231071,NA,1.744157e-02,-0.0005628888,0.018524191
gene13655,0.1347476064,0.0180524702,0.8538925540,0.430197954,0.6391751186,0.388220940,NA,-6.575867e-04,0.6989559317,0.441630177,⋯,0.720046546,0.1777558770,NA,0.6005555016,0.5665056897,0.4707984924,NA,5.529809e-01,0.0051354270,0.647147500
gene8976,0.0767722795,0.0109018942,0.4301979544,0.625596485,0.3308316543,0.215684834,NA,1.055485e-03,0.4633347023,0.407435834,⋯,0.415068996,0.1340194310,NA,0.3228043463,0.3727710408,0.2970519905,NA,3.096761e-01,0.0017952532,0.405803558
gene11611,0.1272769422,0.0165878932,0.6391751186,0.330831654,0.6937446161,0.354622146,NA,-8.333759e-04,0.5951446164,0.366080918,⋯,0.639706490,0.1427757469,NA,0.5336212604,0.4728305136,0.3341317278,NA,5.124991e-01,0.0018105673,0.601817418
gene8827,0.0658413906,0.0102039461,0.3882209399,0.215684834,0.3546221458,0.470721242,NA,1.633390e-04,0.3918141602,0.219309619,⋯,0.397210397,0.1116328655,NA,0.3396006959,0.2913751138,0.1586797719,NA,3.039342e-01,0.0014933732,0.322841237
gene15880,NA,NA,NA,NA,NA,NA,NaN,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
gene7778,-0.0018104406,-0.0008876778,-0.0006575867,0.001055485,-0.0008333759,0.000163339,NA,2.588419e-01,0.0004126148,0.002924443,⋯,-0.000999930,-0.0012138399,NA,0.0008678653,-0.0003855775,0.0014090549,NA,6.584503e-05,-0.0012448128,-0.000502525
gene12686,0.1726072263,0.0223713578,0.6989559317,0.463334702,0.5951446164,0.391814160,NA,4.126148e-04,0.8665166132,0.506315012,⋯,0.778044690,0.2562425132,NA,0.6814217526,0.5618239053,0.4199021065,NA,6.349315e-01,0.0024742488,0.688057873
gene30953,0.1166067118,0.0133132339,0.4416301767,0.407435834,0.3660809183,0.219309619,NA,2.924443e-03,0.5063150116,0.625396324,⋯,0.517328009,0.0826996542,NA,0.3944776059,0.3900783043,0.3997065100,NA,4.397651e-01,0.0014105444,0.476194226


In [33]:
cor2[idxs, idxs]

,gene26973,gene12715,gene13655,gene8976,gene11611,gene8827,gene15880,gene7778,gene12686,gene30953,⋯,gene9236,gene3718,gene31243,gene30532,gene4360,gene30984,gene15086,gene3158,gene11397,gene11612
gene26973,0.1663417251,-0.0005491162,-0.0096824364,-0.0040786076,0.0144140475,-4.094936e-03,NaN,-5.916221e-04,0.0021441088,-0.0022025536,⋯,-0.022589853,2.143121e-03,NaN,-7.331538e-03,0.0031713197,-0.016651958,NaN,-0.0015689407,-0.0012338461,0.003583199
gene12715,-0.0005491162,0.2977867772,-0.0065887798,-0.0021851954,-0.0017938024,-9.189790e-04,NaN,-2.524180e-05,0.0170970322,-0.0030596324,⋯,-0.027216959,-1.257107e-04,NaN,-3.263486e-03,0.0232254909,-0.009028093,NaN,-0.0014637508,-0.0006104408,-0.001273618
gene13655,-0.0096824364,-0.0065887798,0.3027285997,0.0123062120,0.0258495291,3.443267e-03,NaN,-7.503097e-03,-0.0002533101,-0.0050893142,⋯,0.015751041,-1.763370e-04,NaN,6.235336e-03,0.0139142677,0.015372302,NaN,-0.0144974725,0.0070465720,-0.000114624
gene8976,-0.0040786076,-0.0021851954,0.0123062120,0.1551404109,-0.0014833290,2.266840e-03,NaN,-2.521496e-03,0.0205607865,0.0336993676,⋯,-0.010340689,-3.872615e-03,NaN,-2.620067e-03,0.0071111564,0.013487617,NaN,0.0009680584,-0.0023695437,0.007419347
gene11611,0.0144140475,-0.0017938024,0.0258495291,-0.0014833290,0.1256632717,3.396426e-04,NaN,-2.137886e-03,-0.0094122981,-0.0101910672,⋯,-0.013182712,7.572166e-04,NaN,1.045059e-03,0.0012909082,-0.020959532,NaN,-0.0023044724,-0.0035952559,0.005638199
gene8827,-0.0040949360,-0.0009189790,0.0034432670,0.0022668404,0.0003396426,1.220941e-01,NaN,-9.980862e-04,-0.0015843983,-0.0084827007,⋯,0.011477709,-1.016410e-03,NaN,-5.092852e-05,0.0092866967,-0.010356471,NaN,-0.0025356730,-0.0015644422,-0.003134062
gene15880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gene7778,-0.0005916221,-0.0000252418,-0.0075030968,-0.0025214962,-0.0021378859,-9.980862e-04,NaN,3.933411e-01,-0.0053033602,0.0212579163,⋯,-0.006895018,3.903162e-05,NaN,-3.640576e-03,-0.0007924965,-0.004934337,NaN,-0.0013097734,-0.0009279042,-0.001509772
gene12686,0.0021441088,0.0170970322,-0.0002533101,0.0205607865,-0.0094122981,-1.584398e-03,NaN,-5.303360e-03,0.1182613651,0.0294019706,⋯,-0.004251396,4.102978e-03,NaN,8.546268e-03,0.0038253899,0.059169208,NaN,-0.0027570539,-0.0064656580,0.005668234
gene30953,-0.0022025536,-0.0030596324,-0.0050893142,0.0336993676,-0.0101910672,-8.482701e-03,NaN,2.125792e-02,0.0294019706,0.1475247400,⋯,-0.012198882,-2.111754e-03,NaN,1.498985e-03,0.0026103842,0.103991177,NaN,-0.0003052057,-0.0032863319,0.005929791


In [30]:
cor1[5, 5]

[1] 0.3631495

In [ ]:
write.csv(saver.cor, '/data_volume/memento/saver/saver_corrs.csv')

### Sample from saver posterior

In [4]:
saver_obj <- readRDS('/data_volume/memento/saver/melanoma_dropseq_saver.rds')

In [9]:
dim(saver_obj[['estimate']])

[1]   16 8498

In [15]:
sample = sample.saver(saver_obj, rep=1, seed=50)

In [16]:
sample

,CTCGCGAGTAGC,CGGAGGCACTCG,GCAAGTCGATAT,GGACAATTTGTA,TGACAATTGACC,TAAGACTTCCCT,GAGGAAGGACTC,GAAACGGACAGA,TCGATTGGAGAA,ATCTAGTCCCCA,⋯,AGCCCTGACAAC,ACTCTCGATTCC,GGTCAAATAAGA,ACCTCCCCTATA,ACCTCCCCTACC,CCATTTTTTCCT,TAAAGCGTGTAC,GATCAGAAGGTA,AGCGAGACGATG,ATTCTTGTGTAC
BABAM1,0.081,0.026,0.008,0.121,0.072,0.071,0.040,0.065,0.066,0.077,⋯,0.089,0.145,0.064,0.062,0.211,0.042,0.069,0.088,0.088,0.188
C1S,0.033,0.002,0.000,0.003,0.000,0.002,0.033,0.002,0.000,0.000,⋯,0.001,0.000,0.000,0.000,0.068,0.005,0.000,0.000,0.000,0.000
CCNA2,0.029,0.005,0.079,0.033,0.152,0.013,0.043,0.016,0.038,0.042,⋯,0.062,0.009,0.154,0.254,0.107,0.054,0.026,0.104,0.073,0.004
FGFR1,0.001,0.001,0.003,0.000,0.002,0.006,0.001,0.001,0.065,0.005,⋯,0.000,0.000,0.000,0.006,0.012,0.008,0.014,0.000,0.066,0.013
FOSL1,0.003,0.105,0.174,0.000,0.022,0.000,0.334,0.029,0.057,0.157,⋯,0.016,0.001,0.000,0.029,0.011,0.011,0.099,0.001,0.119,0.174
GAPDH,12.591,3.966,6.537,11.876,9.765,8.087,8.045,10.472,16.591,13.323,⋯,9.503,9.207,4.853,6.163,9.667,11.476,12.979,15.315,4.904,9.659
JUN,0.000,0.002,0.242,0.000,0.000,0.006,0.000,0.109,0.000,0.000,⋯,0.037,0.000,0.003,0.015,0.000,0.051,0.000,0.000,0.000,0.002
KDM5A,0.186,0.071,0.032,0.563,0.200,0.225,0.045,0.515,0.379,0.298,⋯,0.250,0.190,0.216,0.180,0.056,0.415,0.319,0.279,0.291,0.340
KDM5B,0.060,0.279,0.018,0.009,0.047,0.229,0.095,0.009,0.206,0.004,⋯,0.024,0.000,0.111,0.075,0.040,0.005,0.126,0.016,0.046,0.124
LMNA,0.464,0.293,0.616,0.474,0.850,0.247,0.300,1.173,0.655,0.826,⋯,0.622,1.136,0.813,1.198,0.481,0.832,0.579,0.516,0.159,0.671
